In [ ]:
# Make a data directory inside the current directory
import os
os.makedirs("./data", exist_ok=True)

In [ ]:
# Download the dataset
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition

In [ ]:
# Unzip the folder
!unzip fruit-and-vegetable-image-recognition.zip -d ./data/

In [ ]:
# Import Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Paths to dataset directories
base_dir = '/home/safwannazir911/Desktop/data'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating batches of image data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
# Define the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

# Load the saved weights
model.load_weights('/home/safwannazir911/Desktop/food_classification_model/model.weights.h5')

# Model summary
model.summary()


In [ ]:
# Load the entire model 
model_path = '/home/safwannazir911/Desktop/food_classification_model'
model = load_model(model_path)

# Model summary
model.summary()

In [ ]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the model
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


In [ ]:
# Evaluate the model
eval_results = model.evaluate(test_generator, verbose=1)

# Print the results
print(f"Test Loss: {eval_results[0]}")
print(f"Test Accuracy: {eval_results[1] * 100:.2f}%")


In [ ]:
#Save the model
model.save('food_classification_model.keras')

# Save weights with the correct filename extension
model.save_weights('food_classification_weights.weights.h5')


In [13]:
# Updated calorie map with actual average values
calorie_map = {
    'Banana': 89,
    'Apple': 52,
    'Pear': 39,
    'Grapes': 69,
    'Orange': 47,
    'Kiwi': 61,
    'Watermelon': 30,
    'Pomegranate': 83,
    'Pineapple': 50,
    'Mango': 60,
    'Cucumber': 16,
    'Carrot': 41,
    'Capsicum': 20,
    'Onion': 40,
    'Potato': 77,
    'Lemon': 29,
    'Tomato': 18,
    'Radish': 16,
    'Beetroot': 43,
    'Cabbage': 25,
    'Lettuce': 15,
    'Spinach': 23,
    'Soybean': 446,
    'Cauliflower': 25,
    'Bell Pepper': 20,
    'Chilli Pepper': 40,
    'Turnip': 28,
    'Corn': 96,
    'Sweetcorn': 86,
    'Sweet Potato': 86,
    'Paprika': 282,
    'Jalapeño': 29,
    'Ginger': 80,
    'Garlic': 149,
    'Peas': 81,
    'Eggplant': 25
}

# Load your CNN model
cnn_model = load_model('/home/safwannazir911/Desktop/food_classification_model')

def predict_food_class(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    predictions = cnn_model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]

    class_labels = list(train_generator.class_indices.keys())
    predicted_class = class_labels[predicted_class_index]

    return predicted_class


def estimate_calories(predicted_class):
    # Convert predicted class to title case to match the format in calorie_map
    predicted_class_title = predicted_class.title()

    # Fetch the calories from the map
    calories = calorie_map.get(predicted_class_title, None)

    if calories is None:
        return "Unknown food class"
    return calories

In [14]:
# Example usage
img_path = '/home/safwannazir911/Desktop/grapes.jpg'
predicted_class = predict_food_class(img_path)
print(f"Predicted Food Class: {predicted_class}")

estimated_calories = estimate_calories(predicted_class)
print(f"Estimated Calories: {estimated_calories}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 689ms/step
Predicted Food Class: grapes
Estimated Calories: 69
